In [1]:
# Import of basic libraries
import tensorflow as tf
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from builtins import range, input
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image

In [2]:
# Import extra libraries
import os
from pathlib import Path
from PIL import Image
from tensorflow import keras
from random import shuffle

base_model = keras.models.load_model("densenet201_ct.h5")

class FeatureExtractor:
    def __init__(self):
        base_model = keras.models.load_model("densenet201_ct.h5")
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)
    def extract(self, img):
        # Resize the image
        img = img.resize((224, 224))
        # Convert the image color space
        img = img.convert('RGB')
        # Reformat the image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # Extract Features
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)
    
if __name__ == '__main__':
    fe = FeatureExtractor()

In [3]:
# Import the libraries
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Variables
fe = FeatureExtractor()
features = []
features_cam = []
img_paths = []
image_paths = []

alpha = 0.8

first_tier_sum = 0
first_tier_sum_2 = 0
total_nn = 0
total_nn_2 = 0

# Extract the feautures of the images
for feature_path in Path(".\\kaggle\\val_set_kaggle").glob("*.npy"):
    features.append(np.load(feature_path))
    img_paths.append(Path(".\\kaggle\\val_set_kaggle") / (feature_path.stem + ".png"))
features = np.array(features) 
    
for feature_path_cam in Path(".\\kaggle\\grad_cam_images").glob("*.npy"):
    features_cam.append(np.load(feature_path_cam))
features_cam = np.array(features_cam) 

# For each image calculate the 'Nearest Neighbor' and 'First Tier' metrics
for image_path in img_paths:
    # Insert the image query
    img = Image.open(image_path)
    # Extract its features
    query = fe.extract(img)

    # Calculate the similarity (distance) between images
    dists = np.linalg.norm(features - query, axis=1)
    # Extract 5 images that have lowest distance
    ids = np.argsort(dists)[:5]
    scores = [(dists[id], img_paths[id]) for id in ids]

    features_mod = features * features_cam
    query_mod = query * features_cam

    # Calculate the similarity (distance) between images
    dists_2 = np.linalg.norm(features_mod - query_mod, axis = 1)
    # Extract 5 images that have lowest distance
    ids_2 = np.argsort(dists_2)[:5]
    scores_2 = [(dists_2[id], img_paths[id]) for id in ids_2]

    dists_final = dists + alpha * dists_2
    # Extract 5 images that have lowest distance
    ids_final = np.argsort(dists_final)[:5]
    scores_final = [(dists_final[id], img_paths[id]) for id in ids_final]

    images = [(img_paths[id]) for id in ids]
    images = [str(image) for image in images]

    images_final = [(img_paths[id]) for id in ids_final]
    images_final = [str(image) for image in images_final]

    non_covid = [img for img in images if os.path.basename(img).startswith("Non-Covid")]
    covid = [img for img in images if img not in non_covid]

    non_covid_final = [img for img in images_final if os.path.basename(img).startswith("Non-Covid")]
    covid_final = [img for img in images_final if img not in non_covid_final]

    relevant_images = non_covid if os.path.basename(image_path).startswith("Non-Covid") else covid
    relevant_images_final = non_covid_final if os.path.basename(image_path).startswith("Non-Covid") else covid_final
    
    basename_prefix = os.path.basename(image_path).split(" ")[0]
    
    # Nearest Neighbor- Simple CBIR
    nn = 0
    if os.path.basename(images[1]).startswith(basename_prefix):
        nn += 1
    total_nn += nn
    
    # Nearest Neighbor-CBIR/Grad-CAM
    nn_2 = 0
    if os.path.basename(images_final[1]).startswith(basename_prefix):
        nn_2 += 1
    total_nn_2 += nn_2
    
    # First tier - Simple CBIR
    first_tier = len(relevant_images) / 5
    first_tier_sum += first_tier
    
    # First tier - CBIR/Grad-CAM
    first_tier_2 = len(relevant_images_final) / 5
    first_tier_sum_2 += first_tier_2

# Print the final results
print('Nearest Neighbor - Simple CBIR:', round(total_nn / len(glob.glob('.\\kaggle\\val_set_kaggle\\*.png')),4))
print('Nearest Neighbor - CBIR/Grad-CAM:', round(total_nn_2 / len(glob.glob('.\\kaggle\\val_set_kaggle\\*.png')),4))
print('First four - Simple CBIR:', round(first_tier_sum / len(glob.glob('.\\kaggle\\val_set_kaggle\\*.png')), 4))
print('First four - CBIR/Grad-CAM:', round(first_tier_sum_2 / len(glob.glob('.\\kaggle\\val_set_kaggle\\*.png')), 4))

1/1 [==============================] - 0s 51ms/step
Nearest Neighbor - Simple CBIR: 0.9713
Nearest Neighbor - CBIR/Grad-CAM: 0.9713
First four - Simple CBIR: 0.9409
First four - CBIR/Grad-CAM: 0.9403
